In [ ]:
# Example Notebook for DeepLabCut(DLC) manual ingestion steps
from workflow.pipeline.dlc import insert_new_dlc_model
from workflow.pipeline import reference

In [ ]:
# Instruction for ingesting new models into the model.Model() table

# Set Project Path to specify the location of the config.yaml file
# This is the only required paramter
example_project_path = "/home/ubuntu/workflows/sciops-dev_sabatini/inbox/dlc_projects/Testing2D-Kim-2022-01-04"

# None of the following need to be specified, all are optional parameters
model_prefix = "dlc_test_"
model_description = "Test Model Ingestion for DLC"
paramset_idx = 1

insert_new_dlc_model(project_path=example_project_path, 
                    model_prefix=model_prefix, 
                    paramset_idx=paramset_idx, 
                    model_description=model_description)


In [ ]:
# Manual step to insert Camera(s) into reference.Device()
# This step is required for the model.VideoRecording table to be populated

# For example 
device_id=0,
device_name='camera1', 
device_description='dlc test camera'
reference_dict = dict(device_id=device_id,
                    device_name=device_name, 
                    device_description=device_description)
reference.Device.insert1(reference_dict)